In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
dirProc = 'data/processed/'

arrestsDirty = pd.read_csv((dirProc + 'arrestdata-all.csv'), index_col='Unnamed: 0')
homeGames = pd.read_csv((dirProc + 'iowa-home-games.csv'))

/home/user/.anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# helper function
def toDate(x):
    return datetime.strptime(x, '%Y-%m-%d')

# get set of dates
gameDates = homeGames['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d')).tolist()
homeGames['Date'] = homeGames['Date'].apply(lambda x: toDate(x))

In [4]:
# get game information 
def getLookup(x):
    #check if theres a day at all
    if (x['arrweekday']):
        
        # Calc day
        day = x['arrweekday']
        weekday = 1 if day in ['Friday', 'Saturday', 'Sunday'] else 0

        if (weekday == 1):
            if (day == 'Saturday'):
                lookup = toDate(x['arrdate'])
            elif (day == 'Friday'):
                lookup = toDate(x['arrdate']) + timedelta(days=1)
            else:
                lookup = toDate(x['arrdate']) - timedelta(days=1)  
            return (lookup)


In [5]:

def getGame(x):
    
    #check if theres a day at all
    if (x['arrweekday']):
        
        # Calc day
        day = x['arrweekday']
        weekday = 1 if day in ['Friday', 'Saturday', 'Sunday'] else 0

        if (weekday == 1):
            lookup = getLookup(x)
    
            gameWeekend = 1 if (lookup in gameDates) else 0
            return gameWeekend
        else:
            return 0
    # return 0 if theres no day
    else:
        return 0
    
def getDelta(x):
    #check if its a gameweekend
    if (x['gameWeekend'] == 1):
        lookup = getLookup(x)
        try:
            gameDelta = homeGames[homeGames['Date'] == lookup]['Delta'].iat[0]
        except:
            gameDelta = ''
        return gameDelta
    else:
        return False

def getTeam(x):
    #check if its a gameweekend
    if (x['gameWeekend'] == 1):
        lookup = getLookup(x)
        gameTeam = homeGames[homeGames['Date'] == lookup]['Visitor'].iat[0]
        return gameTeam
    
def getWin(x):
    #check if its a gameweekend
    if (x['gameWeekend'] == 1):
        lookup = getLookup(x)
        gameWin = homeGames[homeGames['Date'] == lookup]['Win'].iat[0]
        return gameWin

In [6]:
arrestsDirty['gameWeekend'] = arrestsDirty.apply(lambda x: getGame(x), axis=1)
arrestsDirty['lookup'] = arrestsDirty.apply(lambda x: getLookup(x), axis=1)
arrestsDirty['gameDelta'] = arrestsDirty.apply(lambda x: getDelta(x), axis=1)
arrestsDirty['gameTeam'] = arrestsDirty.apply(lambda x: getTeam(x), axis=1)
arrestsDirty['gameWin'] = arrestsDirty.apply(lambda x: getWin(x), axis=1)

In [7]:
arrestsDirty.to_csv((dirProc + 'arrests-games-comb.csv'), index=False)

In [10]:
arrestsDirty.iloc[1000:1050]

,dob,pstreet,pcity,arrtime,arrdate,arrstreet,arrstatus,charge,age,driving,arryear,arrweekday,key,gameWeekend,gameDelta,gameTeam,gameWin
1001,1989-07-03,302 patch st,leighton,16.766667,2008-09-04,burlington st/gilbert st,on view,interference,19.239011,0,2008.0,Thursday,1001,0,False,None,NaN
1002,1989-12-14,845 cross park av 1b,iowa city,17.116667,2008-09-04,burlington st/gilbert st,on view,operating while intoxicated w/accident,18.788462,0,2008.0,Thursday,1002,0,False,None,NaN
1003,1987-01-10,1490 deerfield dr w,north liberty,5.316667,2009-09-04,burlington st/gilbert st,on view,public intoxication,22.728022,0,2009.0,Friday,1003,1,1,Northern Iowa,1.0
1004,1991-03-03,25 byington rd c324,iowa city,12.266667,2009-09-06,burlington st/gilbert st,summoned/cited,interference,18.576923,0,2009.0,Sunday,1004,1,1,Northern Iowa,1.0
1005,1986-12-28,1257 florence ave,clinton,9.716667,2007-09-09,burlington st/gilbert st,on view,operating while under the influence,20.769231,0,2007.0,Sunday,1005,1,35,Syracuse,1.0
1006,1986-12-28,1257 florence ave,clinton,9.716667,2007-09-09,burlington st/gilbert st,on view,driving while license revoked,20.769231,0,2007.0,Sunday,1006,1,35,Syracuse,1.0
1007,1966-08-17,225 sugar creek ln c13,north liberty,10.066667,2007-09-09,burlington st/gilbert st,on view,operating while under the influence,41.203297,0,2007.0,Sunday,1007,1,35,Syracuse,1.0
1008,1986-10-01,168c 290 st,west branch,23.400000,2006-12-17,burlington st/governor st,on view,leaving the scene of personal injury acc,20.280220,0,2006.0,Sunday,1008,0,False,None,NaN
1009,1984-07-27,1000 oakcrest st 103,iowa city,8.366667,2008-05-15,burlington st/governor st,on view,operating while under the influence,23.881868,0,2008.0,Thursday,1009,0,False,None,NaN
1010,1984-09-14,332 s linn st 202,iowa city,9.150000,2006-08-15,burlington st/governor st,summoned/cited,public intoxication,21.991758,0,2006.0,Tuesday,1010,0,False,None,NaN
